<a href="https://colab.research.google.com/github/vrushaligirkar/NLP-Project-QA-system/blob/master/Demo_SQuAD_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Ref: https://towardsdatascience.com/question-answering-with-bert-xlnet-xlm-and-distilbert-using-simple-transformers-4d8785ee762a
import json
import os
!pip install simpletransformers
from simpletransformers.question_answering import QuestionAnsweringModel

!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
!sh setup.sh

from google.colab import drive

     |████████████████████████████████| 163kB 6.4MB/s 
     |████████████████████████████████| 3.8MB 19.3MB/s 
     |████████████████████████████████| 573kB 42.2MB/s 
     |████████████████████████████████| 204kB 48.2MB/s 
     |████████████████████████████████| 1.0MB 42.4MB/s 
     |████████████████████████████████| 890kB 49.9MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=72695cda859e5f84a4deda60c2f23ad7f6f2c301e22f4f57ad5af0154830e73d
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=6a21c5c3a807b01c9700f2f845224623d6aec4997bcf1ccca36fcccc0446e836
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built seqeval sacremoses
ERROR: transformers 2.8.0 has requirement tokenizers==0.5.2, but you'll have tokenizers 0.7.0 which

In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Load SQuAD data
training_file = '/content/gdrive/My Drive/Colab Notebooks/data_SQuAD_V2.0/train-v2.0.json'
dev_file = '/content/gdrive/My Drive/Colab Notebooks/data_SQuAD_V2.0/dev-v2.0.json'

In [0]:
# Prepare the SQuAD data
with open(training_file,'r') as f:
  train_data = json.load(f)

train_data = [item for topic in train_data['data'] for item in topic['paragraphs'] ]

In [0]:
with open(dev_file, 'r') as f:
  dev_data = json.load(f)

dev_data = [item for topic in dev_data['data'] for item in topic['paragraphs'] ]

In [6]:
dev_data[0:1]

[{'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
  'qas': [{'answers': [{'answer_start': 159, 'text': 'France'},
     {'answer_start': 159, 'text': 'France'},
     {'answer_start': 159, 'text': 'France'},
     {'answer_start': 159, 'text': 'France'}],
    'id': '56ddde6b9a695914005b9628',


In [0]:
output_dir = '/content/gdrive/My Drive/Colab Notebooks/data_SQuAD_V2.0/BERT/outputs/'
best_model_dir = '/content/gdrive/My Drive/Colab Notebooks/data_SQuAD_V2.0/BERT/outputs/best_model/'
cache_dir = '/content/gdrive/My Drive/Colab Notebooks/data_SQuAD_V2.0/BERT/cache/'

train_args = {
    'output_dir': output_dir,
    'cache_dir': cache_dir,
    'best_model_dir': best_model_dir,
    'fp16': True,

    'learning_rate': 3e-5,
    'num_train_epochs': 2,
    'max_seq_length': 384,
    'doc_stride': 128,
    'overwrite_output_dir': True,
    'reprocess_input_data': False,
    'train_batch_size': 2,
    'gradient_accumulation_steps': 8,
}

In [0]:
# Create the QuestionAnsweringModel
#model = QuestionAnsweringModel('bert', 'bert-base-cased', args=train_args)

# Load already trained model
model = QuestionAnsweringModel('bert', output_dir)

In [9]:
# Making predictions using the model.
to_predict = [
    {
        "context": "This is the context used for demonstrating predictions.",
        "qas": [{"question": "What is this context?", "id": "0"}],
    }
]

print(model.predict(to_predict))

100%|██████████| 1/1 [00:00<00:00, 554.88it/s]



[{'id': '0', 'answer': 'demonstrating predictions'}]


In [10]:
#Code for building Demo
from flask import Flask,request,jsonify
!pip install flask_cors
from flask_cors import CORS

In [0]:
app = Flask(__name__)
CORS(app)
@app.route("/predict",methods=['POST'])
def predict():
    to_predict = [
    {
        "context": "This is the context used for demonstrating predictions.",
        "qas": [{"question": "What is this context?", "id": "0"}],
    }
    ]
    try:
        out = model.predict(to_predict)
        return jsonify({"result":out})
    except Exception as e:
        print(e)
        return jsonify({"result":"Model Failed"})

In [13]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://fcf69e4b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Apr/2020 16:03:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2020 16:03:15] "GET /favicon.ico HTTP/1.1" 404 -
